## Webscraping dos discursos de Bolsonaro

Neste notebook estou desenvolvendo um crawler para coletar os códigos fonte dos links da paginação da webpage dos discursos

### Fonte dos dados:
Site do Planalto: https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos

#### Carregando os pacotes

In [1]:
# Manipulação de dados

import os
import sys
import re
import time
import string
import pickle
import requests
import datetime
import numpy as np
import pandas as pd

#Web scraping e processamento

from collections import Counter
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#### Web Scraping 
Coleta realizada no dia 18 de agosto de 2020

In [2]:
#Primeiro irei armazenar as urls com os discursos em uma lista.
#Criando a lista
lista_urls = []

In [3]:
#Agora farei um loop para criar a lista de urls para fazer 
#o web scraping. 
#Primeiro crio uma lista com os números id das páginas
pgs = list(range(0,270,30))

In [4]:
#Agora crio os loops para salvar as urls criadas em uma lista
for i in pgs:
    url_completa = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=' + str(i)
    lista_urls.append(url_completa)


In [5]:
#Visualizando a lista de urls
lista_urls

['https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=0',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=30',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=60',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=90',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=120',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=150',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=180',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=210',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=240']

In [6]:
#Criando um contador para as paginas. Contei 8 páginas
contador = 8

In [7]:
#Função para fazer o scraping do código fonte das páginas (crawling da
#paginação). Como já sei qual classe html estão os discursos vou
#selecioná-la já na coleta

def scraping_urls(urls):
    
    # Define o driver
    driver = webdriver.Safari()
    
    # Lista para o resultado
    soup_list = []
 
    
    # Contador
    count = 0
    
    # Loop pelas urls
    for i in urls:
        if count < contador:
            driver.get(i)
            driver.refresh()
            time.sleep(5)
            soup_list.append(BeautifulSoup(driver.page_source, 'html.parser').find_all('a', class_='summary url'))
        count += 1
    driver.close()
    return soup_list

In [8]:
#Fazendo o scraping das páginas
fonte= scraping_urls(lista_urls)

In [9]:
#Verificando o código coletado
fonte

[[<a class="summary url" href="https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-a-inauguracao-da-estacao-radar-de-corumba-corumba-ms" title="Document">Discurso do Presidente da República, Jair Bolsonaro, durante a Inauguração da Estação Radar de Corumbá- Corumbá/MS</a>,
  <a class="summary url" href="https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-a-inauguracao-da-usina-termoeletrica-porto-de-sergipe-i-aracaju-se" title="Document">Discurso do Presidente da República, Jair Bolsonaro, durante a Inauguração da Usina Termoelétrica Porto de Sergipe I - Aracaju/SE</a>,
  <a class="summary url" href="https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-cerimonia-de-brevetacao-dos-novos-paraquedistas-rio-de-janeiro-rj" title="Document">Discurso do Presidente da

In [10]:
#Tentando extrair os links das páginas
def extrai_links(soup_object):
    links_list = []
    for s in soup_object:
        links=s.find_all('a', {'href': re.compile("^http://")})
        links_list.append(links)
    return links_list

In [11]:
links= extrai_links(fonte)

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

Estou tendo um problema com o beautiful soup que não reconhece que os elementos dentro das listas são bs4 objects. Assim, não consigo utilizar a função find.all para coletar os href